In [1]:
"""
okay, tasks for this notebook:

-make a teacher with descent class distribution
    --make a student of same architecture, train it (i have all these parts. ez.)
        ---using ints as input (default) and post-sigmoid as output
        ---remove last layer and use logits, then compare with sigmoide added on
        ---both?
    --swap the embedding layer for something that can take random noise.  add that to noiseKD? --on another nb
        ---this should enable us to train on noise, not just int indexes.  compare all
        
"""

'\nokay, tasks for this notebook:\n\n-make a teacher with descent class distribution\n    --make a student of same architecture, train it (i have all these parts. ez.)\n        ---using ints as input (default) and post-sigmoid as output\n        ---remove last layer and use logits, then compare with sigmoide added on\n        ---both?\n    --swap the embedding layer for something that can take random noise.  add that to noiseKD?\n        ---this should enable us to train on noise, not just int indexes.  compare all\n        \n'

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from NoiseKD import Teacher, SimpleLanguageModel, slm_init_config, slm_model_config, count_parameters

In [3]:
SLM = SimpleLanguageModel(**slm_init_config)

In [4]:
count_parameters(SLM)

3105922

In [9]:
slm_init_config['vocab_size']

80

In [6]:
teacher_slm = Teacher(SLM,(160,))

In [ ]:
# for comparison, i need to be starting them off with the same init.  do that.

In [7]:
teacher_slm.configure(**slm_model_config)

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


Configuring Teacher:: 100%|█████████████████████| 50/50 [01:37<00:00,  1.95s/it]

Teacher Configured


In [10]:
args = { 'val_train' : "train"
                      , 'n' : 100_000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_slm.generate_data(**args)

Configuring Teacher:: 100%|█████████████████| 2000/2000 [00:51<00:00, 38.73it/s]


In [ ]:
args = { 'val_train' : "val"
                      , 'n' : 100_000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_slm.generate_data(**args)

In [20]:
student = SimpleLanguageModel(**slm_init_config)

In [21]:
student

SimpleLanguageModel(
  (embedding): Embedding(80, 16)
  (transformer_encoder): TransformerEncoder(
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): Linear(in_features=16, out_features=11, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=11, out_features=16, bias=True)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
       

In [ ]:
##to train form the teacher object was written already

In [22]:
num_epochs = 100
batch_size = 32
learning_rate = 0.00025 #raise this.   this shit is slooooow
momentum = 0.95
criterion = nn.MSELoss()
optimizer = optim.Adam(student.parameters(), lr=learning_rate)

optimizer = optim.SGD(student.parameters(), lr=learning_rate, momentum=momentum)
data = list(zip(teacher_slm.train_inputs, teacher_slm.train_targets))
input_tensors = torch.stack([torch.Tensor(x[0]) for x in data])
target_tensors = torch.stack([torch.Tensor(x[1]) for x in data])
dataset = TensorDataset(input_tensors, target_tensors)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#conventional train, repeats throught the dataset

print_every = 1 #its working, its just lots of data mama.

losses = []  # List to store losses

for epoch in range(num_epochs):
    total_loss = 0
    for input_batch, target_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        output = student(input_batch)  # Forward pass
        loss = criterion(output, target_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()
        #add early stopping...

    # Print the average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}')

Epoch [1/100], Loss: 0.01218199
Epoch [2/100], Loss: 0.01215971
Epoch [3/100], Loss: 0.01213271
Epoch [4/100], Loss: 0.01209857
Epoch [5/100], Loss: 0.01205400
Epoch [6/100], Loss: 0.01199203
Epoch [7/100], Loss: 0.01189930
Epoch [8/100], Loss: 0.01174310
Epoch [9/100], Loss: 0.01144297
Epoch [10/100], Loss: 0.01098119
Epoch [11/100], Loss: 0.01060179
Epoch [12/100], Loss: 0.01033759
Epoch [13/100], Loss: 0.01018839
Epoch [14/100], Loss: 0.01007457
Epoch [15/100], Loss: 0.01000466
Epoch [16/100], Loss: 0.00993143
Epoch [17/100], Loss: 0.00986348
Epoch [18/100], Loss: 0.00983032
Epoch [19/100], Loss: 0.00981814
Epoch [20/100], Loss: 0.00981172
Epoch [21/100], Loss: 0.00980867
Epoch [22/100], Loss: 0.00980651
Epoch [23/100], Loss: 0.00980508
Epoch [24/100], Loss: 0.00980358
Epoch [25/100], Loss: 0.00980265
Epoch [26/100], Loss: 0.00980144
Epoch [27/100], Loss: 0.00980076
Epoch [28/100], Loss: 0.00980110
Epoch [29/100], Loss: 0.00980042
Epoch [30/100], Loss: 0.00979980
Epoch [31/100], Los

In [ ]:
#all unique train, 
print_every = 1
batch_eternal = 50
data_n = 5000
num_epochs = 400
losses = []  # List to store losses

for epoch in range(num_epochs):
    total_loss = 0
    teacher.generate_data(
    "train"
    ,data_n
    ,'normal'
    , m =0.0
    , std=1.0
    )
    data = list(zip(teacher.train_inputs, teacher.train_targets))
    input_tensors = torch.stack([torch.Tensor(x[0]) for x in data])
    target_tensors = torch.stack([torch.Tensor(x[1]) for x in data])
    dataset = TensorDataset(input_tensors, target_tensors)
    dataloader = DataLoader(dataset, batch_size=batch_eternal, shuffle=True)
    
    for input_batch, target_batch in dataloader:
        optimizer.zero_grad()  # Zero the gradients
        output = student(input_batch)  # Forward pass
        loss = criterion(output, target_batch)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.8f}')